## Train Ranking Model

In this notebook, we will train a ranking model using gradient boosted trees. 

Let's start by loading the datasets we created in the previous notebook.

In [ ]:
# Uncomment this cell and fill in details if you are running external Python
import os
key=""
with open("api-key.txt", "r") as f:
    key = f.read().rstrip()
os.environ['HOPSWORKS_PROJECT']="hm"
os.environ['HOPSWORKS_HOST']="35.240.81.237"
os.environ['HOPSWORKS_API_KEY']=key

In [ ]:
import hopsworks

project = hopsworks.login()
fs = project.get_feature_store()

In [ ]:
dataset_api = project.get_dataset_api()

dataset_api.download("Resources/ranking_train.csv", overwrite=True)
dataset_api.download("Resources/ranking_validation.csv", overwrite=True)

In [6]:
import pandas as pd

X_train = pd.read_csv("ranking_train.csv")
X_val = pd.read_csv("ranking_validation.csv")
y_train = X_train.pop("label")
y_val = X_val.pop("label")

X_train.sample(5)

,age,month_sin,month_cos,product_type_name,product_group_name,graphical_appearance_name,colour_group_name,perceived_colour_value_name,perceived_colour_master_name,department_name,...,user_emb_6,user_emb_7,user_emb_8,user_emb_9,user_emb_10,user_emb_11,user_emb_12,user_emb_13,user_emb_14,user_emb_15
4495257,39.0,1.224647e-16,-1.000000e+00,Skirt,Garment Lower body,Solid,Black,Dark,Black,Conscious Exclusive,...,1.308015,-0.709878,-0.246977,-1.185868,-0.706753,-1.455890,0.379308,1.621206,-1.431212,1.145278
4296597,24.0,-5.000000e-01,8.660254e-01,Trousers,Garment Lower body,Other structure,Blue,Medium,Blue,Trouser,...,0.900624,0.306661,0.751162,-2.542181,-0.220260,0.215980,-0.823136,-1.012750,0.153364,0.671915
2001142,26.0,1.000000e+00,6.123234e-17,Bikini top,Swimwear,Other structure,White,Light,White,Swimwear,...,0.476912,-0.320068,0.760366,-1.888515,-1.773090,-0.598132,-0.121146,-1.526202,0.715934,0.557159
2641345,51.0,-8.660254e-01,-5.000000e-01,Underwear body,Underwear,Solid,Pink,Medium Dusty,Pink,Expressive Lingerie,...,1.001415,-0.343821,0.013374,-0.028308,0.359928,0.001393,0.227801,0.053532,-0.910462,-1.227678
1416006,29.0,1.000000e+00,6.123234e-17,Dress,Garment Full body,Solid,Greenish Khaki,Medium Dusty,Khaki green,Dresses,...,0.214850,-0.055936,1.219609,-0.373164,-1.746445,-1.016426,0.269815,1.240665,0.476183,0.305467


Let's train a model.

In [7]:
from catboost import CatBoostClassifier, Pool

cat_features = list(
    X_train.select_dtypes(include=['string', 'object']).columns
)

pool_train = Pool(X_train, y_train, cat_features=cat_features)
pool_val = Pool(X_val, y_val, cat_features=cat_features)

model = CatBoostClassifier(
    learning_rate=0.2,
    iterations=100,
    depth=10,
    scale_pos_weight=10,
    early_stopping_rounds=5,
    use_best_model=True
)

model.fit(pool_train, eval_set=pool_val)

0:	learn: 0.6477098	test: 0.6570072	best: 0.6570072 (0)	total: 2.92s	remaining: 4m 49s
1:	learn: 0.6168077	test: 0.6350265	best: 0.6350265 (1)	total: 6.25s	remaining: 5m 6s
2:	learn: 0.5918583	test: 0.6179636	best: 0.6179636 (2)	total: 8.74s	remaining: 4m 42s
3:	learn: 0.5733502	test: 0.6063561	best: 0.6063561 (3)	total: 11.1s	remaining: 4m 26s
4:	learn: 0.5564441	test: 0.5962997	best: 0.5962997 (4)	total: 14.1s	remaining: 4m 28s
5:	learn: 0.5408943	test: 0.5876441	best: 0.5876441 (5)	total: 16.5s	remaining: 4m 18s
6:	learn: 0.5229205	test: 0.5764560	best: 0.5764560 (6)	total: 19s	remaining: 4m 12s
7:	learn: 0.5106465	test: 0.5698857	best: 0.5698857 (7)	total: 21.5s	remaining: 4m 6s
8:	learn: 0.4979913	test: 0.5632572	best: 0.5632572 (8)	total: 23.8s	remaining: 4m
9:	learn: 0.4859430	test: 0.5565518	best: 0.5565518 (9)	total: 26.3s	remaining: 3m 56s
10:	learn: 0.4754205	test: 0.5515276	best: 0.5515276 (10)	total: 28.6s	remaining: 3m 51s
11:	learn: 0.4674902	test: 0.5484772	best: 0.5484

Next, we'll evaluate how well the model performs on the validation data.

In [8]:
from sklearn.metrics import classification_report, precision_recall_fscore_support

preds = model.predict(pool_val)

precision, recall, fscore, _ = precision_recall_fscore_support(y_val, preds, average="binary")

metrics = {
    "precision" : precision,
    "recall" : recall,
    "fscore" : fscore
}

print(classification_report(y_val, preds))

              precision    recall  f1-score   support

           0       0.96      0.79      0.87   1756530
           1       0.24      0.68      0.36    175653

    accuracy                           0.78   1932183
   macro avg       0.60      0.73      0.61   1932183
weighted avg       0.90      0.78      0.82   1932183



It can be seen that the model has a low F1-score on the positive class (higher is better). The performance could potentially be improved by adding more features to the dataset, e.g. image embeddings.

Let's see which features our model considers important.

In [9]:
feat_to_score = {feature: score for feature, score in zip(
    X_train.columns, model.feature_importances_)}

feat_to_score = dict(
    sorted(
        feat_to_score.items(),
        key=lambda item: item[1],
        reverse=True
    )
)

feat_to_score


{'item_emb_1': 8.380386180414396,
 'user_emb_1': 6.629093451672942,
 'item_emb_3': 5.743234750003609,
 'item_emb_11': 5.366893617106469,
 'item_emb_10': 4.880974324768764,
 'user_emb_10': 4.709219948736096,
 'user_emb_11': 4.304733336883162,
 'item_emb_2': 3.817951645544647,
 'user_emb_7': 3.682358520343272,
 'user_emb_8': 3.667728763342346,
 'user_emb_3': 3.523555235705989,
 'item_emb_7': 3.4510719462460084,
 'user_emb_9': 3.338411395597691,
 'user_emb_4': 3.298471796625653,
 'item_emb_8': 3.250653251777167,
 'item_emb_15': 2.906185091769092,
 'item_emb_9': 2.674798280867966,
 'item_emb_5': 2.530109853864831,
 'item_emb_6': 2.520830208129553,
 'user_emb_5': 2.4968629504764013,
 'user_emb_15': 2.34001936184849,
 'user_emb_2': 2.2386428074989353,
 'item_emb_4': 2.0120088205283535,
 'user_emb_0': 1.6133374230182345,
 'item_emb_0': 1.5733129467834372,
 'user_emb_6': 1.4553923496964765,
 'month_cos': 1.3695769823410948,
 'user_emb_14': 1.18710666536441,
 'item_emb_14': 1.1623439271789624,


It can be seen that the model places high importance on user and item embedding features. Consequently, better trained embeddings could yield a better ranking model.

Finally, we'll save our model.

In [10]:
import joblib

joblib.dump(model, 'ranking_model.pkl')

### Upload Model to Model Registry

We'll upload the model to the Hopsworks Model Registry.

In [ ]:
import hsml

# connect to Hopsworks Model Registry
conn = hsml.connection()
mr = conn.get_model_registry()

In [ ]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_example = X_train.sample().to_dict("records")
input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema, output_schema)

ranking_model = mr.python.create_model(
    name="ranking_model", metrics=metrics,
    model_schema=model_schema,
    input_example=input_example, description="Ranking model")

ranking_model.save("ranking_model.pkl")

### Next Steps

Now we have trained both a retrieval and a ranking model, which will allow us to generate recommendations for users. In the next notebook, we'll take a look at how we can deploy these models with the `HSML` library.